## Cargamos las librerias

In [ ]:
!pip install gmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmplot
from IPython.display import display

In [ ]:
#cargamos el archivo
from google.colab import files
uploaded = files.upload()

Saving datos_abiertos_2023_mayo.csv to datos_abiertos_2023_mayo (1).csv


## Ajustamos los datos de nuestro dataframe



In [ ]:
# Cargar el archivo CSV en un dataframe
dataframe = pd.read_csv("datos_abiertos_2023_mayo (1).csv")

dataframe['Inicio_del_viaje'] = pd.to_datetime(dataframe['Inicio_del_viaje'])
dataframe['Fin_del_viaje'] = pd.to_datetime(dataframe['Fin_del_viaje'])

# Calcular la duración del viaje en minutos y guardarla en la columna 'Duracion_del_viaje'
dataframe['Duracion_del_viaje'] = (dataframe['Fin_del_viaje'] - dataframe['Inicio_del_viaje']).dt.total_seconds() / 60

mapeo_genero = {'M': 0, 'F': 1} #HOMBRES (M) --> 0 Y MUJERES(F)-->1
# Aplicar el mapeo al dataframe
dataframe['Genero'] = dataframe['Genero'].map(mapeo_genero)
# Obtener el año actual
anio_actual = 2023
# Calcular la edad restando el año de nacimiento del año actual
dataframe['Edad'] = anio_actual - dataframe['Ano_de_nacimiento']
#aqui guardaremos las zonas en las que esta cada estacion
dataframe['Zona_destino'] = dataframe['Destino_Id']
# Verificar la carga exitosa imprimiendo las primeras filas del dataframe
dataframe.head()

,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino
0,28467098,70123,0.0,1967.0,2023-01-05 00:00:00,2023-01-05 00:22:00,64,141,22.0,56.0,141
1,28467099,2237235,0.0,1980.0,2023-01-05 00:00:00,2023-01-05 00:04:00,36,172,4.0,43.0,172
2,28467100,2051727,1.0,2002.0,2023-01-05 00:01:00,2023-01-05 00:10:00,96,296,9.0,21.0,296
3,28467101,2246225,0.0,1969.0,2023-01-05 00:01:00,2023-01-05 00:04:00,33,255,3.0,54.0,255
4,28467102,324247,0.0,1975.0,2023-01-05 00:01:00,2023-01-05 00:13:00,226,231,12.0,48.0,231


## Cargamos un archivo con las id's, latitudes y longitudes de cada una de las estaciones

In [ ]:
uploaded = files.upload()

Saving archivo_texto (1).txt to archivo_texto (1).txt


In [ ]:
#ESTE CODIGO ES PARA DIBUJAR EL MAPA CON LOS PUNTOS

gmap = gmplot.GoogleMapPlotter(20.6597, -103.3496, 13)

id = []
latitudes = []
longitudes = []

with open("archivo_texto (1).txt", "r") as archivo:
    for linea in archivo:
        valores = linea.split(	)  # Separar los valores por espacios en blanco
        id.append(int(valores[0]))
        latitudes.append(float(valores[1]))  # Convertir y guardar el valor de la columna 1
        longitudes.append(float(valores[2]))

# Obtener los datos de latitud y longitu
#  # Ejemplo de longitudes de los puntos

# Marcar los puntos en el mapa
gmap.scatter(latitudes, longitudes, color='red', size=40, marker=True)

# Guardar el mapa en un archivo HTML
gmap.draw("mapa.html")

## Agrupamos por zonas con k-means las estaciones

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
latitudes = np.array(latitudes)[:, np.newaxis]
print(latitudes.shape)
longitudes = np.array(longitudes)[:, np.newaxis]
X = np.concatenate([latitudes, longitudes], axis = 1)

(312, 1)


In [ ]:
# Creamos una instancia de K-means con k clusters
kmeans = KMeans(n_clusters=6)

# Ajustamos el modelo K-means a los datos
kmeans.fit(X)

# Obtenemos las etiquetas de los clusters asignados a cada muestra
labels = kmeans.labels_

# Obtenemos las coordenadas de los centroides
centroids = kmeans.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Calculamos la estacion mas utilizada por los usuarios para iniciar el viaje y utilizamos unicamente los datos de viajes que inician en dicha estacion

In [ ]:
frecuencias = dataframe['Origen_Id'].value_counts()
estacion = frecuencias.index[0]
#Tomamos unicamente los datos tal que la estacion de origen sea la 51
df = dataframe[dataframe['Origen_Id'] == 51]
df.head()

,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino
32,28467130,302357,0.0,1995.0,2023-01-05 00:10:00,2023-01-05 00:16:00,51,9,6.0,28.0,9
44,28467142,2132144,1.0,1991.0,2023-01-05 00:19:00,2023-01-05 00:25:00,51,69,6.0,32.0,69
99,28467197,1061537,0.0,1989.0,2023-01-05 00:50:00,2023-01-05 00:54:00,51,79,4.0,34.0,79
178,28467725,2070201,0.0,2004.0,2023-01-05 06:51:00,2023-01-05 06:57:00,51,86,6.0,19.0,86
208,28467782,1273110,0.0,1995.0,2023-01-05 07:21:00,2023-01-05 07:43:00,51,131,22.0,28.0,131


## Calculamos la zona a la que llega cada uno de los viajes

In [ ]:
mapping = {}
j = 0
for i in id:
  mapping[i] = labels[j]
  j += 1
df['Zona_destino'] = df['Zona_destino'].map(mapping)
df = df.dropna()
df.head()

<ipython-input-124-25c1f98128d0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Zona_destino'] = df['Zona_destino'].map(mapping)


,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino
32,28467130,302357,0.0,1995.0,2023-01-05 00:10:00,2023-01-05 00:16:00,51,9,6.0,28.0,0
44,28467142,2132144,1.0,1991.0,2023-01-05 00:19:00,2023-01-05 00:25:00,51,69,6.0,32.0,0
99,28467197,1061537,0.0,1989.0,2023-01-05 00:50:00,2023-01-05 00:54:00,51,79,4.0,34.0,4
178,28467725,2070201,0.0,2004.0,2023-01-05 06:51:00,2023-01-05 06:57:00,51,86,6.0,19.0,4
208,28467782,1273110,0.0,1995.0,2023-01-05 07:21:00,2023-01-05 07:43:00,51,131,22.0,28.0,2


## Calculamos las horas de salida y duracion de cada uno de los viajes

In [ ]:
#Obtenemos las distintas horas de salida de cada uno de los usuarios
df_Hora1 = df['Inicio_del_viaje'].dt.strftime('%H:%M:%S').str.slice(0, 2)
# Convertir la hora en minutos
df_HoraSalida = df_Hora1.astype(int) * 60 + df['Inicio_del_viaje'].dt.minute

#Obtenemos las distintas horas de llegada de cada uno de los usuarios
df_Hora2 = df['Fin_del_viaje'].dt.strftime('%H:%M:%S').str.slice(0, 2)
# Convertir la hora en minutos
df_HoraLlegada = df_Hora2.astype(int) * 60 + df['Fin_del_viaje'].dt.minute

def cambiar_negativos(value):
  if( value < 0): return 1440 + value

  return value

#Calculamos la duracion del recorrido de cada persona
df_Duracion = df_HoraLlegada - df_HoraSalida
df_Duracion = df_Duracion.apply(cambiar_negativos)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

## Preparamos nuestros predictores de la zona de destino: edad del usuario, duracion del viaje, hora de salida y genero del usuario

In [ ]:
X_data = pd.concat([df['Edad'], df_Duracion, df_HoraSalida, df['Genero']], axis = 1)
features = X_data.columns
X_data = X_data.loc[:, features].values
y_data = df.loc[:, ['Zona_destino']].values

## Ajustamos los modelos de regresion logistica y LDA al conjunto de entrenamiento, calculamos los coeficientes de los modelos y calculamos el score con el conjunto de prueba.

In [ ]:
#separamos los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=18)
#aplicamos regresion logistica
model = LogisticRegression()
model.fit(X_train, y_train)
# Obtener los coeficientes aprendidos
coef = model.coef_[0]
print(coef)
intercept = model.intercept_

# Evaluar el modelo en el conjunto de prueba
score = model.score(X_test, y_test)
print("Exactitud en el conjunto de prueba:", score)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 0.06281634 -0.15192906  0.00305785 -0.00374456]
Exactitud en el conjunto de prueba: 0.6340864440078585


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model = LinearDiscriminantAnalysis()

# Entrenar el clasificador con los datos de entrenamiento
model.fit(X_train, y_train)


coef = model.coef_[0]
print(coef)
# Calcular el puntaje de la predicción
score = model.score(X_test, y_test)

print("Puntaje de la predicción:", score)

[ 0.01345859 -0.07234726  0.00122096  0.02951148]
Puntaje de la predicción: 0.6360510805500982


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
